In [ ]:
pip install Flask flask-cors flasgger

In [ ]:
from flask import Flask, request, jsonify, render_template
from flask_cors import CORS
from flasgger import Swagger
import sqlite3

def create_app():
    app = Flask(__name__)
    CORS(app)
    Swagger(app)

    app.config['CORS_HEADERS'] = 'Content-Type'
    app.config['CORS_RESOURCES'] = {r"/*": {"origins": "*"}}
    app.config['CORS_SUPPORTS_CREDENTIALS'] = True

    @app.after_request
    def after_request(response):
        response.headers.add('Access-Control-Allow-Origin', '*')
        response.headers.add('Access-Control-Allow-Headers', 'Content-Type,Authorization')
        response.headers.add('Access-Control-Allow-Methods', 'GET,POST,PUT,DELETE,OPTIONS')
        return response

    @app.route('/')
    def index():
        """Welcome Endpoint
        ---
        responses:
          200:
            description: Welcome message
        """
        return jsonify({'message': 'Welcome to the SQLite API!'})

    @app.route('/home')
    def webapp_home():
        """Web App Home
        ---
        responses:
          200:
            description: Render the web app home page
        """
        return render_template('index.html')

    @app.route('/health', methods=['GET'])
    def health_check():
        """Health Check Endpoint
        ---
        responses:
          200:
            description: API health status
        """
        return jsonify({'status': 'healthy'})

    @app.route('/tables', methods=['GET'])
    def get_tables():
        """Get All Tables
        ---
        responses:
          200:
            description: List of all tables in the database
        """
        return jsonify(execute_query("SELECT name FROM sqlite_master WHERE type='table';", fetch_all=True))

    @app.route('/table/<string:name>', methods=['GET'])
    def get_table(name):
        """Get Table Data
        ---
        parameters:
          - name: name
            in: path
            type: string
            required: true
            description: Name of the table
        responses:
          200:
            description: Data from the specified table
        """
        return jsonify(fetch_table_data(name))

    @app.route('/schemas', methods=['GET'])
    def get_schemas():
        """Get Table Schemas
        ---
        responses:
          200:
            description: Schema of all tables
        """
        return jsonify(fetch_schemas())

    @app.route('/search', methods=['GET'])
    def search():
        """Search Data
        ---
        parameters:
          - name: query
            in: query
            type: string
            required: true
            description: Search query
        responses:
          200:
            description: Search results
        """
        query = request.args.get('query')
        return jsonify(search_data(query))

    @app.route('/data', methods=['GET', 'POST'])
    def handle_data():
        """Handle Data
        ---
        get:
          description: Get all data
          responses:
            200:
              description: List of all data
        post:
          description: Insert new data
          parameters:
            - name: body
              in: body
              required: true
              schema:
                type: object
                properties:
                  Name:
                    type: string
                  Email:
                    type: string
                  Phone:
                    type: string
          responses:
            201:
              description: Data inserted successfully
        """
        if request.method == 'GET':
            return jsonify(fetch_table_data('user_profile'))
        elif request.method == 'POST':
            data = request.get_json()
            insert_data(data)
            return jsonify({'status': 'success'}), 201

    @app.route('/data/<int:id>', methods=['GET', 'PUT', 'PATCH', 'DELETE'])
    def handle_data_by_id(id):
        """Handle Data by ID
        ---
        get:
          description: Get data by ID
          parameters:
            - name: id
              in: path
              type: integer
              required: true
              description: ID of the data
          responses:
            200:
              description: Data for the specified ID
        put:
          description: Update data by ID
          parameters:
            - name: id
              in: path
              type: integer
              required: true
              description: ID of the data
            - name: body
              in: body
              required: true
              schema:
                type: object
                properties:
                  Name:
                    type: string
                  Email:
                    type: string
                  Phone:
                    type: string
          responses:
            200:
              description: Data updated successfully
        patch:
          description: Partially update data by ID
          parameters:
            - name: id
              in: path
              type: integer
              required: true
              description: ID of the data
            - name: body
              in: body
              required: true
              schema:
                type: object
                additionalProperties:
                  type: string
          responses:
            200:
              description: Data updated successfully
        delete:
          description: Delete data by ID
          parameters:
            - name: id
              in: path
              type: integer
              required: true
              description: ID of the data
          responses:
            200:
              description: Data deleted successfully
        """
        if request.method == 'GET':
            return jsonify(fetch_data_by_id(id))
        elif request.method == 'PUT':
            data = request.get_json()
            update_data(id, data)
            return jsonify({'status': 'success'})
        elif request.method == 'PATCH':
            data = request.get_json()
            patch_data(id, data)
            return jsonify({'status': 'success'})
        elif request.method == 'DELETE':
            delete_data(id)
            return jsonify({'status': 'success'})

    return app

def connect_db():
    return sqlite3.connect('sqlite.db')

def execute_query(query, params=(), fetch_all=False):
    conn = connect_db()
    cur = conn.cursor()
    cur.execute(query, params)
    result = cur.fetchall() if fetch_all else cur.fetchone()
    conn.commit()
    cur.close()
    conn.close()
    return result

def fetch_table_data(table_name):
    conn = connect_db()
    cur = conn.cursor()
    cur.execute(f"SELECT * FROM {table_name}")
    data = cur.fetchall()
    columns = [column[0] for column in cur.description]
    cur.close()
    conn.close()
    return [dict(zip(columns, row)) for row in data]

def fetch_schemas():
    conn = connect_db()
    cur = conn.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cur.fetchall()
    schemas = {}
    for table in tables:
        table_name = table[0]
        cur.execute(f"PRAGMA table_info({table_name})")
        schemas[table_name] = cur.fetchall()
    cur.close()
    conn.close()
    return schemas

def search_data(query):
    return execute_query(
        "SELECT * FROM user_profile WHERE Name LIKE ? OR Email LIKE ?",
        ('%' + query + '%', '%' + query + '%'),
        fetch_all=True
    )

def fetch_data_by_id(id):
    conn = connect_db()
    cur = conn.cursor()
    cur.execute("SELECT * FROM user_profile WHERE Id=?", (id,))
    data = cur.fetchone()
    columns = [column[0] for column in cur.description]
    cur.close()
    conn.close()
    return dict(zip(columns, data)) if data else {}

def insert_data(data):
    execute_query(
        "INSERT INTO user_profile (Name, Email, Phone) VALUES (?, ?, ?);",
        (data['Name'], data['Email'], data['Phone'])
    )

def update_data(id, data):
    execute_query(
        "UPDATE user_profile SET Name=?, Email=?, Phone=? WHERE Id=?;",
        (data['Name'], data['Email'], data['Phone'], id)
    )

def patch_data(id, data):
    update_fields = ", ".join([f"{key}=?" for key in data.keys()])
    update_values = list(data.values()) + [id]
    execute_query(f"UPDATE user_profile SET {update_fields} WHERE Id=?;", tuple(update_values))

def delete_data(id):
    execute_query("DELETE FROM user_profile WHERE Id=?", (id,))

if __name__ == '__main__':
    app = create_app()
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000


Press CTRL+C to quit
127.0.0.1 - - [20/Apr/2025 22:50:49] "GET /apidocs/ HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2025 22:50:49] "GET /flasgger_static/swagger-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [20/Apr/2025 22:50:49] "GET /flasgger_static/swagger-ui-bundle.js HTTP/1.1" 304 -
127.0.0.1 - - [20/Apr/2025 22:50:49] "GET /flasgger_static/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
127.0.0.1 - - [20/Apr/2025 22:50:49] "GET /flasgger_static/lib/jquery.min.js HTTP/1.1" 304 -
127.0.0.1 - - [20/Apr/2025 22:50:49] "GET /apispec_1.json HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2025 22:50:49] "GET /flasgger_static/favicon-32x32.png HTTP/1.1" 304 -
127.0.0.1 - - [20/Apr/2025 22:51:00] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2025 22:51:08] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2025 22:51:10] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2025 22:51:10] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2025 22:51:12] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2025 22:51:12] "